# Hemibrain Connectome Query

By Charles Xu @ Caltech

We will query the Hemibrain connectome to obtain connectivity matrices by select single cells or cell types in relavent neuropils.

## Setup

In [1]:
# import caveclient
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import colors
from scipy.io import savemat
import pickle
import sympy as sp
import os
import csv

### User defined variables

In [2]:
# User options
savedata = True                                            # Set to True to save data to files
datapath = '/Users/charlesxu/local/data/mcp/hemibrain/v1.2' # MBA

plot = True                                                # Set to True to plot
showplots = True                                           # Set to True to show plots
saveplots = False                                           # Set to True to save plots to files
plotpath = '/Users/charlesxu/local/data/mcp/hemibrain/plots' # MBA

## Query by cell type

In [3]:
# Read the cell types CSV file
cell_types_df_path = os.path.join(datapath, 'traced-neurons.csv') # Path to cell types CSV file
cell_types_df = pd.read_csv(cell_types_df_path)
cell_types_df.head()  # Display the first few rows

,bodyId,type,instance
0,200326126,CL141,CL141_L
1,202916528,FS4C,FS4C(FQ12)_C4_R
2,203253072,FS4A,FS4A(AB)(FQ12)_C3_R
3,203253253,SMP505,SMP505(PDM22)_L
4,203257652,FS4C,FS4C(FQ12)_C6_R


In [4]:
# Read the sparse connectivity CSV file
connections_df_path = os.path.join(datapath, 'traced-total-connections.csv') # Path to connectivity CSV file
connections_df = pd.read_csv(connections_df_path)
connections_df.head()  # Display the first few rows

,bodyId_pre,bodyId_post,weight
0,294437328,295470623,1
1,294437328,295133902,1
2,294437328,448260940,1
3,294437328,294783423,1
4,294437328,5812979995,1


In [5]:
ctoi = ['Delta7', 'EPG'] # Cell types of interest

### Query for cell ID

In [32]:
# Identify cell IDs for cell types of interest
ctoi_list = []
noi_list = []
for ctype in ctoi:
    ids = cell_types_df[cell_types_df['type'] == ctype]['bodyId'].tolist()
    cts = [cell_types_df[cell_types_df['type'] == ctype]['type'].iloc[0]] * len(ids)
    noi_list.extend(ids)
    ctoi_list.extend(cts)
    print(f"Found {len(ids)} neurons of type {ctype}")

print(f"Number of neurons of interest: {len(noi_list)}")
print(f"First few cell types of interest: {ctoi_list}")
print(f"First few neuron IDs: {noi_list[:10]}")

Found 42 neurons of type Delta7
Found 46 neurons of type EPG
Number of neurons of interest: 88
First few cell types of interest: ['Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'Delta7', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG', 'EPG']
First few neuron IDs: [734581598, 880530613, 880875736, 880880259, 881221166, 910442723, 910442752, 910442782, 910783731, 910783

In [42]:
if savedata:
    try:
        with open(os.path.join(datapath, 'noi_list.csv'), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Neuron ID'])  # Add a header
            for neuron_id in noi_list:
                writer.writerow([neuron_id])
    except Exception as e:
        print(f"Error saving noi_list.csv: {e}")
        
    try:
        with open(os.path.join(datapath, 'ctoi_list.csv'), 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['Cell Type'])  # Add a header
            for cell_type in ctoi_list:
                writer.writerow([cell_type])
    except Exception as e:
        print(f"Error saving ctoi_list.csv: {e}")

### Query for weights

In [36]:
filtered_connections_df = connections_df[
    (connections_df["bodyId_pre"].isin(noi_list)) &
    (connections_df["bodyId_post"].isin(noi_list))
]
print(f"Number of filtered connections: {len(filtered_connections_df)}")
filtered_connections_df.head()

Number of filtered connections: 5170


,bodyId_pre,bodyId_post,weight
229804,387364605,789126240,1
229813,387364605,912545106,3
229815,387364605,1035045015,1
229818,387364605,911565419,1
229821,387364605,911138168,1


## Construct connectivity matrix

In [37]:
# Manually assign weights to neurotransmitters
# These weights are used to scale the number of synapses between neurons
ctoi_nt = {
    'Delta7': 'GABA',
    'EPG': 'ACH'
}

nt_weights = {
    'GABA': -1, # References?
    'ACH': 1,
    'GLU': -1,
    'DA': 1,
    'SER': 1,
    'OCT': 1,
    'Other': 0
}

In [38]:
# Create a mapping from bodyId to cell type
bodyid_to_type = cell_types_df.set_index('bodyId')['type'].to_dict()

# Map bodyId_pre to cell type
filtered_connections_df['cell_type'] = filtered_connections_df['bodyId_pre'].map(bodyid_to_type)

# Map cell type to neurotransmitter
filtered_connections_df['nt_type'] = filtered_connections_df['cell_type'].map(ctoi_nt)

filtered_connections_df

/var/folders/xr/smwtgd4n7zs6_vpwrvw58_m40000gn/T/ipykernel_90715/129991923.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_connections_df['cell_type'] = filtered_connections_df['bodyId_pre'].map(bodyid_to_type)
/var/folders/xr/smwtgd4n7zs6_vpwrvw58_m40000gn/T/ipykernel_90715/129991923.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_connections_df['nt_type'] = filtered_connections_df['cell_type'].map(ctoi_nt)


,bodyId_pre,bodyId_post,weight,cell_type,nt_type
229804,387364605,789126240,1,EPG,ACH
229813,387364605,912545106,3,EPG,ACH
229815,387364605,1035045015,1,EPG,ACH
229818,387364605,911565419,1,EPG,ACH
229821,387364605,911138168,1,EPG,ACH
...,...,...,...,...,...
3459261,5813080838,911129339,24,EPG,ACH
3459263,5813080838,880880259,23,EPG,ACH
3459266,5813080838,911565419,29,EPG,ACH
3459277,5813080838,911578496,35,EPG,ACH


In [40]:
filtered_connections_df.loc[:, 'connectivity'] = filtered_connections_df['weight'] * filtered_connections_df['nt_type'].map(nt_weights)
filtered_connections_df

,bodyId_pre,bodyId_post,weight,cell_type,nt_type,connectivity
229804,387364605,789126240,1,EPG,ACH,1
229813,387364605,912545106,3,EPG,ACH,3
229815,387364605,1035045015,1,EPG,ACH,1
229818,387364605,911565419,1,EPG,ACH,1
229821,387364605,911138168,1,EPG,ACH,1
...,...,...,...,...,...,...
3459261,5813080838,911129339,24,EPG,ACH,24
3459263,5813080838,880880259,23,EPG,ACH,23
3459266,5813080838,911565419,29,EPG,ACH,29
3459277,5813080838,911578496,35,EPG,ACH,35


In [41]:
# Group the connections by neuron pair
grouped_connections_df = filtered_connections_df.groupby(['bodyId_pre', 'bodyId_post']).agg({'connectivity': 'sum'}).reset_index()

# Connectivity matrix
n_neurons = len(noi_list)
connectivity_matrix = np.zeros((n_neurons, n_neurons))
for _, row in grouped_connections_df.iterrows():
    pre_id = row['bodyId_pre']
    post_id = row['bodyId_post']
    conn = row['connectivity']
    pre_idx = noi_list.index(pre_id)
    post_idx = noi_list.index(post_id)
    connectivity_matrix[pre_idx, post_idx] = conn

del grouped_connections_df

if savedata:
    connectivity_df = pd.DataFrame(connectivity_matrix, index=noi_list, columns=noi_list)
    connectivity_df.to_csv(os.path.join(datapath, 'connectivity_matrix.csv'))

connectivity_matrix

array([[  0.,  -7.,  -9., ...,   0.,   0.,  -3.],
       [ -5.,   0., -24., ...,  -1.,  -2.,   0.],
       [ -5., -25.,   0., ...,  -2.,   0.,   0.],
       ...,
       [  9.,  12.,  17., ...,   0.,   0.,   0.],
       [ 19.,  32.,  16., ...,   1.,   0.,   0.],
       [  2.,   2.,   4., ...,   0.,   0.,   0.]], shape=(88, 88))